## Prticle swarm algorithm

In [ ]:
import numpy as np
from itertools import cycle
import xml.etree.ElementTree as ET
MIN_DURATION,MAX_DURATION = 5 , 50
CYCLE_LENGTH = 7

class particule():
    def __init__(self,d):
        self.pos = np.random.uniform(MIN_DURATION,MAX_DURATION,(2,CYCLE_LENGTH))
        self.Pbest = self.pos
        self.Pbestcost = float('inf')
        self.d = d
        self.speed =np.random.uniform(-12,-12, (2,CYCLE_LENGTH))

    def setPos(self,):
        
        self.pos = np.clip(self.speed + self.pos,MIN_DURATION,MAX_DURATION)

    def setSpeed(self,w,alpha,beta,Gbest):
        self.speed = np.clip(w* self.speed + alpha * np.random.rand()*(self.Pbest - self.pos) + beta * np.random.rand()*(Gbest - self.pos)\
                             ,-5,5)
        
    def setPbest(self,score):
        self.Pbest = self.pos
        self.Pbestcost = score
        


## Simulation on double intersections' network

In [2]:
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import optparse
import random

if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

from sumolib import checkBinary  # noqa
import traci  # noqa



import traci.constants as tc
def run(particle):
    """execute the TraCI control loop"""
    step = 0
    
    network = ET.parse('data2/double.net.xml')
    nets = network.findall('tlLogic') 
    dur = [34,6,31,6] 
    j = 0
    for signal in nets:
        #print(j)
        i = 0
        for phase in signal.iter('phase'):
            #print('phase')
            #print(particle.pos.shape)
            phase.set("duration", str(int(particle.pos[j][i])))
            i+=1
            first = False
        j+=1

    network.write("data2/double.net.xml")

    #
    test = cycle([0,1,2,3])
    LastPhase = 0
    s = 0
    
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        step += 1
        if step >=500:
            break
    traci.close()
    sys.stdout.flush()
    xmldoc = ET.parse('tripinfo2.xml')

    tripinfos = xmldoc.findall('tripinfo')

    waiting_time = 0
    for tripinfo in tripinfos:
        waiting_time += float(tripinfo.get('waitingTime'))
    return waiting_time



# this is the main entry point of this script


In [3]:
#options = get_options()
c = cycle(np.arange(10000))
    # this script has been called from the command line. It will start sumo as a
def function(particle):# server, then connect and run
    sumoBinary = checkBinary('sumo')
       # first, generate the route file for this simulation
    #generate_routefile()

        # this is the normal way of using traci. sumo is started as a
        # subprocess and then the python script connects and runs
    traci.start(['sumo', "-c", "data2/cross.sumocfg",
                                 "--tripinfo-output", "tripinfo2.xml"],port =  8873,label=f"{next(c)}")
    
    s = run(particle)
    return s
        

In [6]:
def pso(pop,function,maxiter = 20,alpha =.5, beta = .5 ,w = .9,):
    #initialiser les swarms
    swarms = []
    for i in range(pop):
        #initial_position = np.random.choice([0,1], CYCLE_LENGTH)
        swarms.append(particule(CYCLE_LENGTH))
        #print(swarms[i].pos)
    #initialiser Gbest
    
    costGbest = float('inf')
    Gbest = np.zeros(CYCLE_LENGTH)
    #Gfeatures = float('inf')
    history = []
    while maxiter:
        #a= np.zeros((pop,d+1))
        for i in range(pop):
            swarms[i].setSpeed(w,alpha,beta,Gbest) 
            
            swarms[i].setPos()
            temp_Res = function(swarms[i])
            if swarms[i].Pbestcost > temp_Res:
                
                swarms[i].setPbest(temp_Res)
                if costGbest > swarms[i].Pbestcost:
                    costGbest = swarms[i].Pbestcost
                    Gbest = swarms[i].Pbest
        if maxiter%1 ==0:
            print('best pos  is : ' ,Gbest)
            print('the best time is :',costGbest)
            
          
        maxiter-=1
    return costGbest,Gbest,history

In [7]:
pso(20,function)

best pos  is :  [[32.79174988 41.6150344  23.73775494  5.         26.24356565 38.41415742
  29.02309771]
 [ 5.45689883 10.22925212 39.67086246 10.71410894 18.13839484 14.52628876
   5.        ]]
the best time is : 27592.0
best pos  is :  [[30.66288823  9.45866769 27.63960511 29.66567188  5.69770221 26.92178565
  14.79954839]
 [ 7.47635169 10.72007144 37.22429557  5.         13.2726384   9.26796077
   5.        ]]
the best time is : 27495.0
best pos  is :  [[31.32627687 10.29770806 31.95069339 20.61396217 10.59785861 27.19435392
  19.35685733]
 [19.63036729  9.88574697 19.43129937  5.         13.13038836 11.8672664
   5.        ]]
the best time is : 26689.0
best pos  is :  [[24.63767089 15.53109085 23.39803209 20.16567188  8.97979609 24.21682495
  15.31323484]
 [ 8.10392551  5.         27.72429557  5.          7.63890003  5.13627158
   5.        ]]
the best time is : 26641.0
best pos  is :  [[24.63767089 15.53109085 23.39803209 20.16567188  8.97979609 24.21682495
  15.31323484]
 [ 8.103

(22894.0,
 array([[32.3214805 , 13.14841215, 13.72306191, 25.88044673, 18.66489741,
         20.36488267, 13.93733561],
        [ 5.        ,  6.63410045, 32.17336976,  5.86466016, 18.34961262,
         22.58348884, 21.81559068]]),
 [])